# Data Preparation

This notebook will organize, clean, and store the data into a parquet file. 

More details on the validity and information of the dataset can be found [here](https://citibikenyc.com/system-data). 

For the purpose of this project, the downloaded files are stored in the `/data` folder. There is another way to collect data from the previous years, however, that can be programmatically done in the near future.

## 1. Combine The Multiple Excel Files

The files that were downloaded are typically in a `.zip`.

The program below locates the local `/data` folder and the Excel files within. It will combine and save the raw data as a parquet file to be used for further data preprocessing.

In [61]:
import math
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import warnings
warnings.filterwarnings('ignore')

In [9]:
import os
from datetime import datetime

path = os.getcwd()
data_path = os.path.join(path, '..', 'data')
data_files = os.listdir(data_path)

dfs = []
for i, data_file in enumerate(data_files, start=1):

    if (data_file == 'zip') | (data_file == 'saved_data'):
        continue
    
    else:
        trip_data_path = os.path.join(data_path, data_file)
        csv_files = os.listdir(trip_data_path)

        for csv_file in csv_files:
            csv_path = os.path.join(trip_data_path, csv_file)
            dfs.append(pd.read_csv(csv_path, dtype='str'))

    print(f'[{datetime.now().strftime('%d/%m/%Y %H:%M')}]: {i}/{len(data_files)}')

df = pd.concat(dfs)
save_path = os.path.join(data_path, 'saved_data', 'raw_citibike_trips.parquet')
df.to_parquet(save_path)

[14/07/2025 20:15]: 1/7
[14/07/2025 20:15]: 2/7
[14/07/2025 20:15]: 3/7
[14/07/2025 20:15]: 4/7
[14/07/2025 20:16]: 5/7


In [10]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,56BD148A05E26915,electric_bike,2025-01-01 22:19:06.324,2025-01-01 22:23:32.899,W 36 St & 7 Ave,6483.06,W 24 St & 7 Ave,6257.03,40.752149,-73.989539,40.74487634,-73.99529885,member
1,700CCCDF00C08077,electric_bike,2025-01-09 15:20:43.991,2025-01-09 15:26:19.137,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.73829,-73.99006,40.74487634,-73.99529885,member
2,B1A3FAFA5FE651CE,electric_bike,2025-01-01 12:54:26.571,2025-01-01 13:03:03.565,Jackson Ave & 46 Rd,6203.02,Crescent St & Broadway,6827.11,40.74524768,-73.94733276,40.7633589,-73.9286471,member
3,9CB0F28054BA5EBC,electric_bike,2025-01-14 13:43:59.151,2025-01-14 13:49:04.543,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.73829,-73.99006,40.74487634,-73.99529885,member
4,A1B8FDD4FAF0C2B2,electric_bike,2025-01-13 07:20:37.176,2025-01-13 07:29:18.015,Willis Ave & E 143 St,7798.02,1 Ave & E 110 St,7522.02,40.812299,-73.92037,40.7923272,-73.9383,member


## 2. Normalize The RAW Data

Make sure the data is ready for analysis.

This section will manipulate and clean the data. 

In [11]:
null_values = df.isna().sum()
null_values

ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_station_name     5375
start_station_id       5375
end_station_name      38086
end_station_id        39983
start_lat                 0
start_lng                 0
end_lat                2605
end_lng                2605
member_casual             0
dtype: int64

### 2a. Fix Data Types

1. Convert station ids to numeric values
2. Fix date columns to datetime

In [12]:
df['start_station_id'] = pd.to_numeric(df['start_station_id'], errors='coerce')
df['end_station_id'] = pd.to_numeric(df['end_station_id'], errors='coerce')

In [13]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [47]:
df['start_lat'] = pd.to_numeric(df['start_lat'], errors='coerce')
df['start_lng'] = pd.to_numeric(df['start_lng'], errors='coerce')
df['end_lat'] = pd.to_numeric(df['end_lat'], errors='coerce')
df['end_lng'] = pd.to_numeric(df['end_lng'], errors='coerce')

### 2b. Create A Stations DataFrame

In [48]:
start_stations = df[
    ['start_station_name', 'start_station_id', 'start_lat', 'start_lng']
    ].drop_duplicates()

end_stations = df[
    ['end_station_name', 'end_station_id', 'end_lat', 'end_lng']
    ].drop_duplicates()

start_stations.rename(
    columns={
        'start_station_name': 'station_name',
        'start_station_id': 'station_id',
        'start_lat': 'lat',
        'start_lng': 'lng'
    },
    inplace=True
)

end_stations.rename(
    columns={
        'end_station_name': 'station_name',
        'end_station_id': 'station_id',
        'end_lat': 'lat',
        'end_lng': 'lng'
    },
    inplace=True
)

stations_df = pd.concat([start_stations, end_stations]).dropna()
stations_df.set_index('station_id', inplace=True)
stations_df.to_parquet('../data/saved_data/citibike_stations.parquet')
stations_df.head()

,station_name,lat,lng
station_id,,,
6483.06,W 36 St & 7 Ave,40.752149,-73.989539
5980.11,Broadway & E 19 St,40.738290,-73.990060
6203.02,Jackson Ave & 46 Rd,40.745248,-73.947333
7798.02,Willis Ave & E 143 St,40.812299,-73.920370
8126.07,Grand Concourse & E 167 St,40.834263,-73.917641


### 2c. Fill Or Drop Missing Values

In [49]:
stations_dict = stations_df.to_dict()
station_names = stations_dict['station_name']
station_lats, station_lngs = stations_dict['lat'], stations_dict['lng']

df['start_station_name'] = df['start_station_id'].map(station_names)
df['end_station_name'] = df['end_station_id'].map(station_names)

df['end_lat'] = df['end_station_id'].map(station_lats)
df['end_lng'] = df['end_station_id'].map(station_lngs)

In [ ]:
perct_dropped = (df.isna().sum().max())/len(df)
print(f'{round(perct_dropped * 100, 2)}% of records dropped from the DataFrame')
df.dropna(inplace=True)

0.0% of records dropped from the DataFrame


### 2d. Replace Textual Data

In [27]:
df['rideable_type'] = df['rideable_type'].str.replace('_bike', '').str.title()
df['member_casual'] = df['member_casual'].str.title()

## Feature Engineering

- Trip duration (seconds)
- Distance traveled (miles)

### 3a. Calculate Trip Duration

In [39]:
df['trip_duration'] = (df['ended_at'] - df['started_at']).dt.seconds

### 3b. Add Distance Traveled ([Haversine Formula](https://www.geeksforgeeks.org/dsa/haversine-formula-to-find-distance-between-two-points-on-a-sphere/))

In [51]:
def haversine(lat1, lon1, lat2, lon2):
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0

    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c * 0.6213711922

df['trip_distance'] = df.apply(
    lambda x:
    haversine(
        x['start_lat'],
        x['start_lng'],
        x['end_lat'],
        x['end_lng']
    ),
    axis=1
)

## 4. Save The Data

In [55]:
df = df[60 * 120 >= df['trip_duration']]

df.drop_duplicates(inplace=True)
df.set_index('ride_id', inplace=True)
df.to_parquet('../data/saved_data/clean_citibike_trips.parquet')

C:\Users\kazir\AppData\Local\Temp\ipykernel_23196\1594007788.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


KeyError: "None of ['ride_id'] are in the columns"

In [56]:
df.head(2)

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,trip_distance
ride_id,,,,,,,,,,,,,,
56BD148A05E26915,Electric,2025-01-01 22:19:06.324,2025-01-01 22:23:32.899,W 36 St & 7 Ave,6483.06,W 24 St & 7 Ave,6257.03,40.752149,-73.989539,40.744876,-73.995299,Member,266,0.586000
700CCCDF00C08077,Electric,2025-01-09 15:20:43.991,2025-01-09 15:26:19.137,Broadway & E 19 St,5980.11,W 24 St & 7 Ave,6257.03,40.738290,-73.990060,40.744876,-73.995299,Member,335,0.531323


## 5. Reverse Geocoding Stations

[Credit](https://towardsdatascience.com/reverse-geocoding-with-nyc-bike-share-data-cdef427987f8/)

In [60]:
geo = Nominatim(user_agent="kazirshahria@gmail.com")
reverse = RateLimiter(geo.reverse, min_delay_seconds=1, max_retries=2)

In [66]:
def reverse_geocode(lat, long):
    coordinate = f'{lat}, {long}'
    response = reverse(coordinate)
    address = response.raw['address']
    neighborhood = address.get('neighbourhood')
    borough = address.get('borough')
    county = address.get('county')
    city = address.get('city')
    state = address.get('state')
    postcode = address.get('postcode')
    return (neighborhood, borough, county, city, state, postcode)

values = stations_df.apply(
    lambda col: 
    reverse_geocode(
        col['lat'], 
        col['lng']
        ), axis=1
)

stations_df[
    ['neighborhood', 'borough', 'county', 'city', 'state', 'postcode']
    ] = values.apply(pd.Series)

stations_df.head()

KeyboardInterrupt: 

### 5a. Save Data

In [ ]:
stations_df.to_parquet('../data/saved_data/clean_citibike_stations.parquet')